1. Map/merge the 23andme file and the variant-drug annotation file based on dbSNP_ID (also known as rsID).

The merged result should have the following columns:

dbSNP_ID, GENE_SYMBOL, DRUG_NAME, PMID, PHENOTYPE_CATEGORY, SIGNIFICANCE, NOTES, SENTENCE, ALLELE_PharmGKB (variant alleles in annotation), ALLELE_23andme (variant alleles in 23andme file)

In [5]:
import pandas as pd

_23andme_v5 = pd.read_csv("23andme_v5_hg19_ref.txt", sep="\t", header = None, names = ["CHR", "POS", "dbSNP_ID", "ALLELE_23andme"])

var_drug_ann = pd.read_csv("var_drug_ann.tsv", sep="\t")

#renaming to match the assignment naming conventions
var_drug_ann.rename(columns={
    "Variant/Haplotypes": "dbSNP_ID",
    "Gene": "GENE_SYMBOL",
    "Drug(s)": "DRUG_NAME",
    "Phenotype Category": "PHENOTYPE_CATEGORY",
    "Significance": "SIGNIFICANCE",
    "Notes": "NOTES",
    "Sentence": "SENTENCE",
    "Alleles": "ALLELE_PharmGKB"
}, inplace=True)

#just keeping the columns we need
var_drug_ann = var_drug_ann[[
    "dbSNP_ID",
    "GENE_SYMBOL",
    "DRUG_NAME",
    "PMID",
    "PHENOTYPE_CATEGORY",
    "SIGNIFICANCE",
    "NOTES",
    "SENTENCE",
    "ALLELE_PharmGKB"
]]

merged = pd.merge(var_drug_ann, _23andme_v5[["dbSNP_ID", "ALLELE_23andme"]], on="dbSNP_ID", how="inner")
merged.head()

,dbSNP_ID,GENE_SYMBOL,DRUG_NAME,PMID,PHENOTYPE_CATEGORY,SIGNIFICANCE,NOTES,SENTENCE,ALLELE_PharmGKB,ALLELE_23andme
0,rs706795,FAIM2,"citalopram, escitalopram, fluoxetine, fluvoxam...",40054571,Efficacy,no,"""We observed nominally significant association...",Allele T is associated with increased response...,T,T
1,rs16918842,OPRK1,heroin,31940240,Dosage,no,No significant difference in allele or genotyp...,Allele T is not associated with dose of heroin...,T,C
2,rs163184,KCNQ1,sitagliptin,39792745,Efficacy,yes,"""KCNQ1 gene polymorphisms also significantly a...",Genotype GG is associated with decreased respo...,GG,T
3,rs7754840,CDKAL1,sitagliptin,39792745,Efficacy,no,"""Patients with the rs7754840 CG genotype showe...",Genotype CG is associated with increased respo...,CG,G
4,rs1799853,CYP2C9,sitagliptin,39792745,Efficacy,yes,"""CYP2C9 gene polymorphisms also significantly ...",Genotype TT is associated with decreased respo...,TT,C


2. Filter the output so that it only contains significant associations (SIGNIFICANCE is yes) for variants that affect the drug efficacy (`PHENOTYPE_CATEGORY` is `efficacy`).

In [7]:
filtered = merged[
    (merged["SIGNIFICANCE"].str.lower() == "yes") &
    (merged["PHENOTYPE_CATEGORY"].str.lower() == "efficacy")
].reset_index(drop=True)
filtered.head()


,dbSNP_ID,GENE_SYMBOL,DRUG_NAME,PMID,PHENOTYPE_CATEGORY,SIGNIFICANCE,NOTES,SENTENCE,ALLELE_PharmGKB,ALLELE_23andme
0,rs163184,KCNQ1,sitagliptin,39792745,Efficacy,yes,"""KCNQ1 gene polymorphisms also significantly a...",Genotype GG is associated with decreased respo...,GG,T
1,rs1799853,CYP2C9,sitagliptin,39792745,Efficacy,yes,"""CYP2C9 gene polymorphisms also significantly ...",Genotype TT is associated with decreased respo...,TT,C
2,rs7903146,TCF7L2,exenatide,30700996,Efficacy,yes,"""After treatment with exenatide, only CT/TT in...",Genotypes CT + TT is associated with increased...,CT + TT,C
3,rs8099917,IFNL3,"peginterferon alfa-2a, peginterferon alfa-2b, ...",26075078,Efficacy,yes,A multivariate logistic model showed that the ...,Genotype TT is associated with increased respo...,TT,T
4,rs8099917,IFNL3,"peginterferon alfa-2b, ribavirin",22328925,Efficacy,yes,This genotype is associated with sustained vir...,Genotype TT is associated with increased respo...,TT,T
